In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
test_set = pd.read_csv('x_test.csv')

In [72]:
test_set.head(10).T 

,0,1,2,3,4,5,6,7,8,9
Unnamed: 0,0,1,2,3,4,5,6,7,8,9
Brand,Toyota,Hyundai,Land,Mazda,Volkswagen,Hyundai,Subaru,Audi,Ford,Isuzu
Year,2005.0,2015.0,2017.0,2004.0,2023.0,2018.0,2018.0,2018.0,2014.0,2023.0
Model,HiAce,Tucson,Rover,3,Golf,I30,Outback,S5,Territory,FSR
Car/Suv,Commercial,SUV,SUV,Sedan,NEW Available to order,Hatchback,SUV,Audi Centre Sydney Pre-Owned,SUV,NEW Available to order
Title,2005 Toyota HiAce Slwb,2015 Hyundai Tucson Active X (fwd),2017 Land Rover Discovery Sport TD4 180 SE 5 Seat,2004 Mazda 3 NEO,2023 Volkswagen Golf 110TSI,2018 Hyundai I30 SR,2018 Subaru Outback 2.0D,2018 Audi S5 Sportback Tiptronic Quattro,2014 Ford Territory TS (rwd),2023 Isuzu FSR 140/120-240 (LWB)
UsedOrNew,USED,USED,USED,USED,NEW,USED,USED,USED,USED,NEW
Transmission,Automatic,Automatic,Automatic,Automatic,Automatic,Manual,Automatic,Automatic,Automatic,Automatic
Engine,"4 cyl, 2.5 L","4 cyl, 2 L","4 cyl, 2 L","4 cyl, 2 L","4 cyl, 1.4 L","4 cyl, 1.6 L","4 cyl, 2 L",-,"6 cyl, 2.7 L","4 cyl, 5.2 L"
DriveType,Rear,Front,AWD,Front,Front,Front,AWD,Other,Rear,Rear


In [73]:
test_set.shape

(3337, 19)

In [74]:
test_set = test_set.drop(columns=['Unnamed: 0'])
test_set = test_set.drop(columns=['Title'])
test_set[['Cylinders', 'EngineVolume']] = test_set['Engine'].str.extract(r'(\d+)\s+cyl,\s+([\d.]+)\s+L')
test_set['Cylinders'] = test_set['Cylinders'].astype(float)
test_set['EngineVolume'] = test_set['EngineVolume'].astype(float)
test_set = test_set.drop(columns=['Engine']) 
test_set = test_set.drop(columns=['CylindersinEngine'])
test_set['FuelConsumption'] = test_set['FuelConsumption'].str.extract(r'([\d.]+)\s+L').astype(float)
test_set[['ExteriorColor', 'InteriorColor']] = test_set['ColourExtInt'].str.split('/', n=1, expand=True)
test_set = test_set.drop(columns=['ColourExtInt'])
test_set[['City', 'State']] = test_set['Location'].str.split(', ', expand=True)
test_set = test_set.drop(columns=['Location']) 
test_set['Doors'] = test_set['Doors'].str.extract('(\d+)').astype(float)
test_set['Seats'] = test_set['Seats'].str.extract('(\d+)').astype(float)
test_set.head(10).T 

,0,1,2,3,4,5,6,7,8,9
Brand,Toyota,Hyundai,Land,Mazda,Volkswagen,Hyundai,Subaru,Audi,Ford,Isuzu
Year,2005.0,2015.0,2017.0,2004.0,2023.0,2018.0,2018.0,2018.0,2014.0,2023.0
Model,HiAce,Tucson,Rover,3,Golf,I30,Outback,S5,Territory,FSR
Car/Suv,Commercial,SUV,SUV,Sedan,NEW Available to order,Hatchback,SUV,Audi Centre Sydney Pre-Owned,SUV,NEW Available to order
UsedOrNew,USED,USED,USED,USED,NEW,USED,USED,USED,USED,NEW
Transmission,Automatic,Automatic,Automatic,Automatic,Automatic,Manual,Automatic,Automatic,Automatic,Automatic
DriveType,Rear,Front,AWD,Front,Front,Front,AWD,Other,Rear,Rear
FuelType,Diesel,Unleaded,Diesel,Unleaded,Premium,Unleaded,Diesel,Premium,Diesel,Diesel
FuelConsumption,0.0,7.9,5.3,8.9,5.9,7.5,6.3,NaN,8.2,0.0
Kilometres,202406,79956,118977,161876,- / -,40910,90256,39566,228405,- / -


In [75]:
test_set.shape

(3337, 19)

In [76]:
test_set.replace('-', np.nan, inplace=True)
test_set.replace(' -', np.nan, inplace=True)
test_set.replace('- ', np.nan, inplace=True)
test_set.replace(' - ', np.nan, inplace=True)
for col in test_set.columns:
    print(col, '-', test_set[col].isna().sum())

Brand - 0
Year - 0
Model - 0
Car/Suv - 4
UsedOrNew - 0
Transmission - 46
DriveType - 0
FuelType - 124
FuelConsumption - 343
Kilometres - 27
BodyType - 49
Doors - 328
Seats - 344
Cylinders - 359
EngineVolume - 359
ExteriorColor - 82
InteriorColor - 1542
City - 92
State - 92


In [77]:
test_set = test_set.drop(columns=['InteriorColor'])
test_set['Transmission'] = test_set['Transmission'].fillna(test_set.groupby('Model')['Transmission'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['FuelType'] = test_set['FuelType'].fillna(test_set.groupby('Model')['FuelType'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['FuelConsumption'] = test_set['FuelConsumption'].fillna(test_set.groupby('Model')['FuelConsumption'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['Kilometres'] = pd.to_numeric(test_set['Kilometres'], errors='coerce').fillna(np.nan)
test_set['Kilometres'] = test_set['Kilometres'].fillna(test_set.groupby('Year')['Kilometres'].transform(lambda x: x.value_counts().idxmax()))
test_set['BodyType'] = test_set['BodyType'].fillna(test_set.groupby('Model')['BodyType'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['Doors'] = test_set['Doors'].fillna(test_set.groupby('Model')['Doors'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['Seats'] = test_set['Seats'].fillna(test_set.groupby('Model')['Seats'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['Cylinders'] = test_set['Cylinders'].fillna(test_set.groupby('Model')['Cylinders'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['EngineVolume'] = test_set['EngineVolume'].fillna(test_set.groupby('Model')['EngineVolume'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x))
test_set['ExteriorColor'] = test_set['ExteriorColor'].fillna(test_set['ExteriorColor'].value_counts().index[0])
test_set['State'] = test_set['State'].fillna(test_set['State'].value_counts().index[0])
test_set = test_set.drop(columns=['City'])
test_set.isna().sum() 

Brand                0
Year                 0
Model                0
Car/Suv              4
UsedOrNew            0
Transmission         1
DriveType            0
FuelType            20
FuelConsumption    104
Kilometres           0
BodyType             9
Doors               83
Seats              105
Cylinders          101
EngineVolume       101
ExteriorColor        0
State                0
dtype: int64

In [78]:
test_set.shape

(3337, 17)

In [79]:
test_set['FuelConsumption'] = test_set['FuelConsumption'].fillna(test_set['FuelConsumption'].median())
test_set['Doors'] = test_set['Doors'].fillna(test_set['Doors'].median())
test_set['Seats'] = test_set['Seats'].fillna(test_set['Seats'].median())
test_set['Cylinders'] = test_set['Cylinders'].fillna(test_set['Cylinders'].median())
test_set['EngineVolume'] = test_set['EngineVolume'].fillna(test_set['EngineVolume'].median())

In [80]:
test_set.shape

(3337, 17)

In [81]:
test_set.isna().sum() 

Brand               0
Year                0
Model               0
Car/Suv             4
UsedOrNew           0
Transmission        1
DriveType           0
FuelType           20
FuelConsumption     0
Kilometres          0
BodyType            9
Doors               0
Seats               0
Cylinders           0
EngineVolume        0
ExteriorColor       0
State               0
dtype: int64

In [82]:
test_set['Transmission'] = test_set['Transmission'].fillna(test_set['Transmission'].mode()[0])
test_set['FuelType'] = test_set['FuelType'].fillna(test_set['FuelType'].mode()[0])
test_set['BodyType'] = test_set['BodyType'].fillna(test_set['BodyType'].mode()[0])
test_set['Car/Suv'] = test_set['Car/Suv'].fillna(test_set['Car/Suv'].mode()[0])

In [83]:
test_set.shape

(3337, 17)

In [84]:
test_set.isna().sum() 

Brand              0
Year               0
Model              0
Car/Suv            0
UsedOrNew          0
Transmission       0
DriveType          0
FuelType           0
FuelConsumption    0
Kilometres         0
BodyType           0
Doors              0
Seats              0
Cylinders          0
EngineVolume       0
ExteriorColor      0
State              0
dtype: int64

In [85]:
test_set['Brand'].unique()

array(['Toyota', 'Hyundai', 'Land', 'Mazda', 'Volkswagen', 'Subaru',
       'Audi', 'Ford', 'Isuzu', 'Mercedes-Benz', 'Holden', 'Skoda', 'BMW',
       'Mitsubishi', 'Jeep', 'Renault', 'MG', 'Porsche', 'Kia', 'Honda',
       'Nissan', 'LDV', 'Volvo', 'Chevrolet', 'Lexus', 'HSV', 'Tesla',
       'Fiat', 'Bentley', 'Peugeot', 'Suzuki', 'Ssangyong', 'Great',
       'Iveco', 'GWM', 'Jaguar', 'Chrysler', 'Opel', 'Dodge', 'Hino',
       'Ram', 'Mini', 'Maserati', 'International', 'Haval', 'McLaren',
       'Citroen', 'Cupra', 'BYD', 'Aston', 'Daihatsu', 'Foton',
       'Mahindra', 'Alfa', 'Genesis', 'Chery', 'Rover', 'Packard', 'FPV'],
      dtype=object)

In [86]:
test_set['UsedOrNew'].unique()

array(['USED', 'NEW', 'DEMO'], dtype=object)

In [87]:
test_set['Transmission'].unique()

array(['Automatic', 'Manual'], dtype=object)

In [88]:
test_set['DriveType'].unique()

array(['Rear', 'Front', 'AWD', 'Other', '4WD'], dtype=object)

In [89]:
test_set['FuelType'].unique()

array(['Diesel', 'Unleaded', 'Premium', 'Hybrid', 'Electric', 'Other',
       'LPG', 'Leaded'], dtype=object)

In [90]:
test_set['BodyType'].unique()

array(['Commercial', 'SUV', 'Sedan', 'Wagon', 'Hatchback', 'Ute / Tray',
       'Coupe', 'Convertible', 'People Mover', 'Other'], dtype=object)

In [91]:
test_set['ExteriorColor'].unique()

array(['Silver ', 'White ', 'Green ', '5 years ', 'Black ', 'Grey ',
       'Brown ', '6 years ', 'Blue ', 'Gold ', 'Red ', 'Yellow ',
       '3 years ', 'Orange ', '7 years ', '1 years ', 'Beige ',
       '2 years ', 'Burgundy ', 'Bronze ', '4 years '], dtype=object)

In [92]:
test_set['ExteriorColor'] = test_set['ExteriorColor'].str.replace(' ', '')

In [93]:
test_set['ExteriorColor'] = test_set['ExteriorColor'].apply(lambda x: x if x in ['Silver', 'Blue', 'Black', 'Red', 'White', 'Grey', 'Green', 'Brown', 'Gold', 'Beige', 'Orange', 'Yellow', 'Burgundy', 'Bronze'] else 'Other')

In [94]:
test_set['State'].unique()

array(['NSW', 'SA', 'QLD', 'VIC', 'ACT', 'WA', 'NT', 'TAS'], dtype=object)

In [95]:
test_set = test_set.drop(columns=['Seats'])
test_set = test_set.drop(columns=['FuelConsumption'])

In [96]:
ordinalencoder = OrdinalEncoder(categories = [['USED', 'DEMO', 'NEW']])
test_set.loc[:, 'UsedOrNew'] = ordinalencoder.fit_transform(test_set.loc[:, 'UsedOrNew'].to_frame())
ordinalencoder = OrdinalEncoder(categories = [['Manual', 'Automatic']])
test_set.loc[:, 'Transmission'] = ordinalencoder.fit_transform(test_set.loc[:, 'Transmission'].to_frame())
def dum(data, column):
    df = pd.get_dummies(data[column], prefix=column, drop_first=True)
    data = data.drop(column, axis=1)
    return data.join(df)
test_set = dum(test_set, 'DriveType')
test_set = dum(test_set, 'FuelType')
test_set = dum(test_set, 'BodyType')
test_set = dum(test_set, 'ExteriorColor')
test_set = dum(test_set, 'State')
encoder = ce.BinaryEncoder(cols=['Brand'], return_df=True)
test_set = encoder.fit_transform(test_set)
encoder = ce.BinaryEncoder(cols=['Model'], return_df=True)
test_set = encoder.fit_transform(test_set)
test_set = test_set.drop(columns=['Car/Suv'])

In [97]:
test_set

,Brand_0,Brand_1,Brand_2,Brand_3,Brand_4,Brand_5,Year,Model_0,Model_1,Model_2,...,ExteriorColor_Silver,ExteriorColor_White,ExteriorColor_Yellow,State_NSW,State_NT,State_QLD,State_SA,State_TAS,State_VIC,State_WA
0,0,0,0,0,0,1,2005.0,0,0,0,...,True,False,False,True,False,False,False,False,False,False
1,0,0,0,0,1,0,2015.0,0,0,0,...,True,False,False,True,False,False,False,False,False,False
2,0,0,0,0,1,1,2017.0,0,0,0,...,False,True,False,True,False,False,False,False,False,False
3,0,0,0,1,0,0,2004.0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
4,0,0,0,1,0,1,2023.0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,0,0,1,1,1,0,2013.0,0,0,0,...,True,False,False,True,False,False,False,False,False,False
3333,0,0,1,1,1,0,2012.0,0,0,0,...,False,True,False,False,False,True,False,False,False,False
3334,0,1,0,0,1,1,2015.0,0,0,0,...,False,True,False,True,False,False,False,False,False,False
3335,0,0,0,0,1,0,2017.0,0,0,0,...,False,True,False,True,False,False,False,False,False,False


In [98]:
test_set.to_csv('test_set.csv')